In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np
from sympy import Matrix

filename = Path("../img/color/Jayhawk_64x64x3.jpg")

with Image.open(filename, "r") as im:
    display(im)

# True for noisy execution, False for noiseless execution
noisy_execution = False
draw_qc = True


In [ ]:
def display_quantum_circuit(qc):
    if draw_qc:
        display(qc.draw('mpl', reverse_bits=True))
        
def display_matrix(matrix):
    try:
        display(Matrix(matrix))
    except:
        print(matrix)


In [ ]:
from qcc.filters import *

stride = 1
# kernel_raw = avg_filter(3, dim = 2)
# kernel_raw = sobel_filter(3, axis=0)
kernel_raw = laplacian_approx(3)
# kernel_raw = gaussian_blur(3)

npad = tuple((0, 2 ** int(np.ceil(np.log2(N))) - N) for N in kernel_raw.shape)
kernel = np.pad(kernel_raw, pad_width=npad, mode="constant", constant_values=0)
kernel_mag = np.linalg.norm(kernel)

display_matrix(kernel)


In [ ]:
from qcc.quantum import flatten_image, normalize, to_qubits
from qcc.filters import update_dims

psi, *dims = flatten_image(filename, pad=True)
psi, mag = normalize(psi, include_magnitude=True)

dims_q = [to_qubits(x) for x in dims]
dims_out = update_dims(dims, kernel_size=kernel_raw.shape, stride=stride)

num_qubits = sum(dims_q)
num_states = 2**num_qubits

print(dims_out, mag)


In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info.operators import Operator

kernel_shape_q = [int(np.ceil(np.log2(filter_size))) for filter_size in kernel.shape]
num_ancilla = sum(kernel_shape_q)
total_qubits = num_qubits + num_ancilla

qc = QuantumCircuit(total_qubits)
qc.initialize(psi, qc.qubits[:-num_ancilla])

display_quantum_circuit(qc)


In [ ]:
from qcc.quantum.qiskit.convolution import Convolution

qc.compose(Convolution(dims, kernel), inplace=True)
display_quantum_circuit(qc.decompose(Convolution))

In [ ]:
from qiskit import Aer, execute
from qiskit.tools import job_monitor
from qcc.quantum import from_counts

backend = Aer.get_backend('aer_simulator')
shots = backend.configuration().max_shots

if noisy_execution:
    qc.measure_all()
else:
    qc.save_statevector()

job = execute(qc, backend=backend, shots=shots)
job_monitor(job)

result = job.result()

if noisy_execution:
    counts = result.get_counts(qc)
    psi_out = from_counts(counts, shots=shots, num_qubits=total_qubits)
else:
    psi_out = result.get_statevector(qc).data


In [ ]:
from qcc.quantum import reconstruct

i = 0
img = psi_out.data[i*num_states:(i+1)*num_states]
norm = mag * kernel_mag * np.sqrt(2**num_ancilla)

print(min(np.abs(norm*img)), max(np.abs(norm*img)))

img = reconstruct(norm*img, dims, dims_out)
img = Image.fromarray(np.abs(img).astype(np.uint8))
img.save("output.png")

display(img)


### Classical Results

In [ ]:
display_matrix(kernel_raw)


In [ ]:
with Image.open(filename, "r") as im:
    image_data = np.asarray(im, dtype=float)

classical_img = convolution(image_data, kernel_raw, stride=stride)

print(classical_img.shape, min(classical_img.flatten()), max(classical_img.flatten()))

classical_img = classical_img.astype(np.uint8)
classical_img = Image.fromarray(classical_img)
display(classical_img)
classical_img.save("output_classical.png")


In [ ]:
from qcc.quantum import get_fidelity

fid = get_fidelity(np.asarray(classical_img).flatten(), np.asarray(img).flatten())

print(f"Fidelity: {fid:.03%}")
